#### FFD NOTEBOOK

This notebook applies FFD to a geometry and illustrates the maximal deformations corresponding to the given boundaries.

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import random

from aero_optim.geom import split_profile
from aero_optim.ffd.ffd import FFD_2D, RotationWrapper

The plotting function is defined as

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Times",
    "figure.dpi": 300,
    "font.size": 8,
    'legend.fontsize': 8, 
    "axes.titlesize": 8,
    "axes.labelsize": 8
    })

def plot_max_min_profile(ffd: FFD_2D, bounds: tuple[float], ncontrol: int, outdir: str):
    """
    Plots the highest and lowest deformations.
    """
    min_profile = ffd.apply_ffd(np.array([bounds[0]] * 2 * ncontrol))
    upper_min, lower_min = split_profile(min_profile)
    max_profile = ffd.apply_ffd(np.array([bounds[-1]] * 2 * ncontrol))
    upper_max, lower_max = split_profile(max_profile)
    upper_pro, lower_pro = split_profile(ffd.pts)
    width = ffd.max_x - ffd.min_x
    height = ffd.max_y - ffd.min_y
    # Figure
    fig = plt.figure(figsize=(6.5, 4))
    fig.subplots_adjust(hspace=1.5, wspace=2)
    gs = gridspec.GridSpec(2, 4, figure=fig)
    ax1 = plt.subplot(gs[0, 1:3])
    ax2 = plt.subplot(gs[1, :2])
    ax3 = plt.subplot(gs[1, 2:])
    # ax1
    ax1.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
    ax1.plot(min_profile[:, 0], min_profile[:, 1], label=f"min profile", color="b", linewidth=1)
    ax1.plot(max_profile[:, 0], max_profile[:, 1], label=f"max profile", color="r", linewidth=1)
    ax1.fill_between(lower_min[:, 0], lower_min[:, 1], lower_max[:, 1], color="b", alpha=0.1)
    ax1.fill_between(upper_max[:, 0], upper_min[:, 1], upper_max[:, 1], color="r", alpha=0.1)
    # ax2
    delta = ffd.pad_Delta(np.array([bounds[0]] * 2 * ncontrol))
    lat_pts_x = [ffd.from_lat([i / ffd.L, j / ffd.M] + ffd.dPij(i, j, delta))[0] for j in range(ffd.M + 1) for i in range(ffd.L + 1)]
    lat_pts_y = [ffd.from_lat([i / ffd.L, j / ffd.M] + ffd.dPij(i, j, delta))[1] for j in range(ffd.M + 1) for i in range(ffd.L + 1)]
    ax2.add_patch(patches.Rectangle((ffd.x1[0], ffd.x1[1]), width, height, angle=0.0, linewidth=1, edgecolor="grey", facecolor="none", label="lattice"))
    ax2.plot(lat_pts_x[:ncontrol + 2], lat_pts_y[:ncontrol + 2], linewidth=1, linestyle="dashed", marker="s", markersize=5, color="grey")
    ax2.plot(lat_pts_x[-(ncontrol + 2):], lat_pts_y[-(ncontrol + 2):], linewidth=1, linestyle="dashed", marker="s", markersize=5, color="grey")
    ax2.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
    ax2.plot(min_profile[:, 0], min_profile[:, 1], label=f"min profile", color="b", linewidth=1)
    ax2.fill(min_profile[:, 0], min_profile[:, 1], label=f"min profile", color="b", alpha=0.1)
    # ax3
    delta = ffd.pad_Delta(np.array([bounds[-1]] * 2 * ncontrol))
    lat_pts_x = [ffd.from_lat([i / ffd.L, j / ffd.M] + ffd.dPij(i, j, delta))[0] for j in range(ffd.M + 1) for i in range(ffd.L + 1)]
    lat_pts_y = [ffd.from_lat([i / ffd.L, j / ffd.M] + ffd.dPij(i, j, delta))[1] for j in range(ffd.M + 1) for i in range(ffd.L + 1)]
    ax3.add_patch(patches.Rectangle((ffd.x1[0], ffd.x1[1]), width, height, angle=0.0, linewidth=1, edgecolor="grey", facecolor="none", label="lattice"))
    ax3.plot(lat_pts_x[:ncontrol + 2], lat_pts_y[:ncontrol + 2], linewidth=1, linestyle="dashed", marker="s", markersize=5, color="grey")
    ax3.plot(lat_pts_x[-(ncontrol + 2):], lat_pts_y[-(ncontrol + 2):], linewidth=1, linestyle="dashed", marker="s", markersize=5, color="grey")
    ax3.plot(ffd.pts[:, 0], ffd.pts[:, 1], label="baseline profile", color="k", linewidth=1)
    ax3.plot(max_profile[:, 0], max_profile[:, 1], label=f"max profile", color="r", linewidth=1)
    ax3.fill(max_profile[:, 0], max_profile[:, 1], label=f"max profile", color="r", alpha=0.1)
    # plot lattice grid
    ax1.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="a) Design space")
    ax2.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="b) Lower bound deformation")
    ax3.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title="c) Upper bound deformation")
    # legend and display
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "FFD.pdf"), bbox_inches="tight")
    plt.show()

The notebook input variables are:

- `ncontrol` the number of FFD control points on each side of the lattice box
- `bounds` the deformation boundaries
- `file` the path to the file containing the geometry coordinates

**Note**: the number of FFD design variables is `2 * ncontrol`

In [ ]:
ncontrol = 4
bounds = (-0.2, 0.2)
file = "../../examples/LRN-CASCADE/data/lrn_cascade.dat"

In [ ]:
ffd = FFD_2D(file, ncontrol)

The figure below illustrates the geometry and its maximal deformed profiles

In [ ]:
plot_max_min_profile(ffd, bounds, ncontrol, os.getcwd())

Roration wrapper

In [ ]:
ffd_rot = RotationWrapper(ffd)

The figure below illustrates the geometry and random rotations of the baseline

In [ ]:
random.seed(123)
fig, ax = plt.subplots(figsize=(5.2, 3.64))
ax.plot(ffd_rot.pts[:, 0], ffd_rot.pts[:, 1], linestyle="dashed", color="k", label="baseline")
for ii in range(5):
    delta = np.zeros(2 * ncontrol)
    theta = -90 + 180 * random.random()
    profile = ffd_rot.apply_ffd(np.append(delta, theta))
    ax.plot(profile[:, 0], profile[:, 1], label=f"$\\theta={theta:.2f}^\circ$")
delta = np.append(np.ones(ncontrol) * bounds[0], np.ones(ncontrol) * bounds[1])
profile = ffd_rot.apply_ffd(np.append(delta, 0.))
ax.plot(profile[:, 0], profile[:, 1], label=f"$\\theta=0^\circ$")
ax.set(xlabel="$x$ [m]", ylabel="$y$ [m]", title=f"FFD rotated profiles")
ax.legend()